<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-26 15:20:26
-------------------
qualified stocks: 93
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.39 L
Current:  1.36 C
-------------------
Today PnL: -1.44 L (-1.05%)
Current PnL: -19.60 L (-13.33%)
CY Booked + Current PnL: -10.87 L (-7.39%)
-------------------
Total profit:  3.33 L
Total loss:  -22.92 L
-------------------
Total Booked + Current PnL: 16.11 L (13.42%)
Total Booked PnL: 35.71 L (29.75%)
Curr Year Booked PnL: 8.73 L (6.43%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 78.71 L (57.99%)
Deployed:  1.20 C
Current:  1.36 C
CAGR/XIRR %: 7.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301706.0,31359.0,3258.0,2.36,11.60,1.08,12.81,24.0,9.63,2.28,26.18,XY25,NTT,FMCG
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.36,112872.0,8560.0,8093.0,-1.45,8.21,7.17,15.96,9.0,1.06,0.85,26.49,X5K,ATH,METALS
55,MEDANTA,1087.93,1486.00,13.61,H-MC,14.68,162191.0,35991.0,10186.0,0.14,28.52,6.28,36.59,108.0,3.53,1.23,44.66,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.30,H-LC,8.11,171336.0,15159.0,10931.0,-0.80,9.71,6.38,16.71,22.0,1.39,1.29,28.84,X40N,BTT,HEALTHCARE
68,SBIN,760.30,863.00,-13.25,M-LC,4.02,213895.0,12416.0,14802.0,-1.14,6.16,6.92,13.51,60.0,0.84,1.62,19.54,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,KPIGREEN,497.21,731.35,5.18,H-SC,11.07,125446.0,149.0,58859.0,-2.39,0.12,46.92,47.09,141.0,0.0,0.95,57.0,MH,ATH,POWER


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,ITC,411.90,452.0,-38.77,H-LC,3.05,146783.0,-3149.0,17746.0,0.98,-2.10,12.09,9.74,4.0,-0.18,1.11,4.01,X40,NTT,FMCG
53,LTIM,5564.16,7230.2,-4.05,H-LC,3.40,190548.0,-9762.0,69741.0,1.31,-4.87,36.60,29.94,16.0,-0.14,1.44,31.64,X200,ATH,IT
57,NESTLEIND,1134.30,1377.0,-11.91,H-LC,3.42,271159.0,5733.0,51059.0,0.50,2.16,18.83,21.40,11.0,0.11,2.05,8.95,XY25,NTT,FMCG
21,CIPLA,1492.70,1795.0,-19.09,H-LC,4.21,216254.0,11754.0,29670.0,-1.42,5.75,13.72,20.25,10.0,0.40,1.63,14.98,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,6.07,169517.0,-31869.0,119476.0,-0.46,-15.82,70.48,43.51,27.0,-0.27,1.28,18.00,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BRITANNIA,5502.66,6362.67,14.72,H-LC,8.68,207576.0,9480.0,21484.0,3.88,4.79,10.35,15.63,26.0,0.44,1.57,27.68,XY25,ATH,FMCG
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301706.0,31359.0,3258.0,2.36,11.60,1.08,12.81,24.0,9.63,2.28,26.18,XY25,NTT,FMCG
30,GREENPANEL,375.16,537.00,195.22,M-SC,6.01,131736.0,-51342.0,130326.0,1.81,-28.04,98.93,43.14,230.0,-0.39,1.00,23.92,XY24,NTT,MISC
53,LTIM,5564.16,7230.20,-4.05,H-LC,3.40,190548.0,-9762.0,69741.0,1.31,-4.87,36.60,29.94,16.0,-0.14,1.44,31.64,X200,ATH,IT
81,TITAGARH,1097.23,1548.00,-7.14,H-SC,9.62,152372.0,-39643.0,118530.0,1.12,-20.65,77.79,41.08,158.0,-0.33,1.15,28.96,XY24,NTT,ENGINEERING


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,EASEMYTRIP,18.12,26.40,-15.39,M-SC,28.50,81577.0,-95238.0,176035.0,-5.11,-53.86,215.79,45.70,197.0,-0.54,0.62,0.00,XY24,NTT,TRAVEL
71,SIEMENS,4406.90,7969.85,-5.39,H-LC,33.21,76102.0,-34070.0,123141.0,-3.81,-30.92,161.81,80.85,15.0,-0.28,0.58,12.09,AR,ATH,ELECTRICAL
88,VALIANTORG,512.64,838.00,-306.49,H-SC,13.11,87750.0,-45536.0,130133.0,-3.52,-34.16,148.30,63.47,139.0,-0.35,0.66,47.09,XR,NTT,CHEMICALS
60,RAJESHEXPO,517.65,518.00,1617.17,L-SC,8.43,46868.0,-90309.0,90404.0,-3.37,-65.83,192.89,0.07,267.0,-1.00,0.35,16.56,OX40N,NTT,JEWELLERY
33,HAVELLS,1588.02,2069.16,-5.05,H-MC,3.37,235482.0,-12249.0,87317.0,-3.36,-4.94,37.08,30.30,92.0,-0.14,1.78,7.69,X40,ATH,ELECTRICAL


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,94.66,M-SC,8.63,83873.0,-16904.0,17001.0,-2.42,-16.77,20.27,0.09,245.0,-0.99,0.63,10.19,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,15.45,115560.0,-26919.0,46444.0,-2.17,-18.89,40.19,13.70,163.0,-0.58,0.87,36.39,OX40N,NTT,BANKS
20,CERA,7989.07,9475.0,-20.53,H-SC,3.07,113328.0,-30475.0,57219.0,-2.64,-21.19,50.49,18.60,149.0,-0.53,0.86,25.06,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,108.15,M-SC,1.74,136437.0,-38122.0,38161.0,-2.32,-21.84,27.97,0.02,221.0,-1.00,1.03,18.68,OX40N,NTT,PAINTS
50,KANSAINER,299.63,340.0,-69.44,H-SC,3.67,210249.0,-59418.0,95747.0,-0.40,-22.03,45.54,13.47,138.0,-0.62,1.59,6.34,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-25.47,H-LC,15.93,246098.0,-49836.0,143746.0,0.46,-16.84,58.41,31.73,1.0,-0.35,1.86,5.06,X40,BTT,IT
45,INFY,1461.46,2275.00,-16.37,H-LC,11.89,273942.0,12341.0,133273.0,-0.11,4.72,48.65,55.67,3.0,0.09,2.07,11.11,X40,BTT,IT
47,ITC,411.90,452.00,-38.77,H-LC,3.05,146783.0,-3149.0,17746.0,0.98,-2.10,12.09,9.74,4.0,-0.18,1.11,4.01,X40,NTT,FMCG
89,VBL,492.64,671.64,-9.95,H-LC,8.65,273627.0,8587.0,87725.0,-1.43,3.24,32.06,36.33,5.0,0.10,2.07,16.97,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.40,H-LC,12.85,195604.0,-20915.0,113822.0,-1.67,-9.66,58.19,42.91,7.0,-0.18,1.48,2.44,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,13.61,H-MC,14.68,162191.0,35991.0,10186.0,0.14,28.52,6.28,36.59,108.0,3.53,1.23,44.66,XY24,NTT,HEALTHCARE
28,DMART,3764.92,5201.0,-7.35,H-LC,11.15,231324.0,46843.0,23526.0,-0.09,25.39,10.17,38.14,38.0,1.99,1.75,38.69,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,13.61,H-MC,14.68,162191.0,35991.0,10186.0,0.14,28.52,6.28,36.59,108.0,3.53,1.23,44.66,XY24,NTT,HEALTHCARE
15,BLUESTARCO,1646.70,2326.38,7.95,H-MC,4.82,186290.0,21620.0,46349.0,-2.18,13.13,24.88,41.28,89.0,0.47,1.41,22.16,X40N,ATH,AC
51,KPIGREEN,497.21,731.35,5.18,H-SC,11.07,125446.0,149.0,58859.0,-2.39,0.12,46.92,47.09,141.0,0.00,0.95,57.00,MH,ATH,POWER
90,VOLTAS,1278.28,1918.49,-3.74,H-MC,6.76,201840.0,10098.0,85943.0,-0.87,5.27,42.58,50.08,99.0,0.12,1.53,12.42,XY25,ATH,AC
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.03,177653.0,26740.0,30556.0,0.24,17.72,17.20,37.97,93.0,0.88,1.34,47.51,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.03,177653.0,26740.0,30556.0,0.24,17.72,17.20,37.97,93.0,0.88,1.34,47.51,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.34,140869.0,17533.0,116231.0,0.77,14.22,82.51,108.45,119.0,0.15,1.06,39.75,AR,ATH,MISC
91,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,5.56,96375.0,4876.0,73871.0,-1.83,5.33,76.65,86.07,223.0,0.07,0.73,39.42,XR,NTT,DURABLES
92,WIPRO,243.46,420.00,-11.07,M-LC,7.77,156066.0,5121.0,104330.0,-0.84,3.39,66.85,72.51,53.0,0.05,1.18,9.62,XR,NTT,IT
61,RAJOOENG,102.50,143.33,-31.45,H-SC,9.99,103750.0,1250.0,39581.0,-0.19,1.22,38.15,39.83,114.0,0.03,0.78,2.72,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.10,82426.0,-31123.0,71175.0,-0.14,-27.41,86.35,35.27,268.0,-0.44,0.62,101.74,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-4.17,H-SC,16.14,89118.0,-11845.0,106380.0,-2.36,-11.73,119.37,93.63,148.0,-0.11,0.67,32.74,SR,ATH,CHEMICALS
54,MASFIN,326.60,398.61,-19.15,H-SC,12.54,92130.0,-5850.0,27455.0,-1.74,-5.97,29.80,22.05,152.0,-0.21,0.70,33.92,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,7.36,112872.0,8560.0,8093.0,-1.45,8.21,7.17,15.96,9.0,1.06,0.85,26.49,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,83.20,H-SC,15.45,115560.0,-26919.0,46444.0,-2.17,-18.89,40.19,13.70,163.0,-0.58,0.87,36.39,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.10,82426.0,-31123.0,71175.0,-0.14,-27.41,86.35,35.27,268.0,-0.44,0.62,101.74,XR,NTT,HOTELS
28,DMART,3764.92,5201.00,-7.35,H-LC,11.15,231324.0,46843.0,23526.0,-0.09,25.39,10.17,38.14,38.0,1.99,1.75,38.69,X40N,NTT,FMCG
35,HEROMOTOCO,4311.81,5949.07,-3.95,H-MC,14.03,177653.0,26740.0,30556.0,0.24,17.72,17.20,37.97,93.0,0.88,1.34,47.51,AR,ATH,AUTO
37,HINDUNILVR,2413.81,2723.00,-4.12,H-LC,8.23,301706.0,31359.0,3258.0,2.36,11.60,1.08,12.81,24.0,9.63,2.28,26.18,XY25,NTT,FMCG
69,SFL,1016.35,1287.00,39.92,M-SC,10.39,184715.0,-77503.0,147329.0,0.79,-29.56,79.76,26.63,229.0,-0.53,1.40,14.85,XY24,NTT,MISC


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.03
1,25,41.70
2,50,71.40


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.12
LC    36.60
MC    26.28
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.38
X40      14.45
X40N     13.94
XY25     11.97
XR        8.98
AR        8.46
OX40N     7.33
X5K       2.26
MH        1.69
X200      1.44
SR        1.10
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.36
H-MC    23.40
H-SC    22.44
M-SC    13.14
M-LC     7.20
M-MC     2.55
L-SC     1.54
L-LC     1.04
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.78,0.31,30.22
IT,12.12,-16.41,74.55
FINANCE,7.97,-23.35,68.97
BANKS,7.44,-15.98,65.23
MISC,5.94,-16.96,84.00
PAINTS,5.51,-19.75,43.35
ELECTRICAL,4.70,-13.14,62.14
AUTO,4.58,-11.49,59.99
AC,3.67,0.14,33.69


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2882028.0,23
XR,1107137.0,13
AR,1088695.0,10
X40,822806.0,11
X40N,521993.0,11
OX40N,487040.0,10
XY25,463737.0,8
SR,197567.0,2
X5K,138735.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2802901.0,27
M-SC,1507616.0,17
H-MC,1282866.0,18
H-LC,1148222.0,19
M-LC,426560.0,5
M-MC,339954.0,2
L-SC,256748.0,3
L-MC,59510.0,1
L-LC,46626.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      946827.0      6
M-SC       XY24      877193.0      7
H-SC       AR        661668.0      5
           XR        533459.0      6
H-LC       X40       510651.0      6
H-MC       XY24      491296.0      5
M-MC       XY24      339954.0      2
H-LC       AR        236963.0      2
H-MC       X40       226163.0      4
H-SC       OX40N     209217.0      4
H-MC       XY25      201663.0      2
H-SC       SR        197567.0      2
H-LC       X40N      197001.0      5
H-SC       X40N      195304.0      3
M-LC       XY24      176786.0      2
L-SC       XR        166344.0      2
M-SC       AR        159508.0      2
           OX40N     135941.0      4
M-LC       X5K       130642.0      1
H-MC       X40N      129688.0      3
M-SC       XY25      124845.0      1
           XR        124137.0      2
H-MC       XR        119357.0      1
M-LC       XR        104330.0      1
L-SC       OX40N      90404.0      1
M-SC       X40        85992.0      1
H-LC       XY25       75801.0      3
           X200       69741.0      1
L-MC       XR         59510.0      1
H-SC       MH         58859.0      1
H-MC       OX40N      51478.0      1
H-LC       XY24       49972.0      1
L-LC       XY25       46626.0      1
H-MC       MH         32665.0      1
           AR         30556.0      1
M-LC       XY25       14802.0      1
H-LC       X5K         8093.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 42.0 seconds
